# **Groq API Implementation: Conversation Management & JSON Extraction**

This notebook demonstrates two core tasks using the Groq API with the OpenAI SDK compatibility, without relying on any external frameworks.

- Task 1: Managing and summarizing a running conversation history.

- Task 2: Classifying and extracting structured information from user chats into a JSON object.

In [1]:
# Install the required Python libraries
!pip install groq openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.4 MB/s eta 0:00:00


In [25]:
# Import necessary modules
import os
import json
from openai import OpenAI
from getpass import getpass

# --- API Key Configuration ---
# IMPORTANT: It is not secure to hardcode API keys.
# We are doing it here for demonstration purposes only.
# In a production environment, use environment variables or a secret management tool.
# You can get your Groq API key from: https://console.groq.com/keys

GROQ_API_KEY = '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'


# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Initialize the Groq client with OpenAI compatibility
client = OpenAI(
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

# --- Model Configuration ---
# We'll use a smaller model for simple chat and summarization for speed and cost-efficiency.
# A larger, more capable model is better for complex tasks like function calling.
CHAT_MODEL = "llama-3.1-8b-instant"
EXTRACTION_MODEL = "llama-3.3-70b-versatile"

print("✅ Setup complete. Groq client is configured.")

✅ Setup complete. Groq client is configured.


# **Task 1: Managing Conversation History with Summarization**

This task involves maintaining a conversation history, implementing various truncation strategies, and periodically summarizing the chat to keep the context concise for the model.

# 1. Helper Functions
We'll define functions for truncation and summarization.

In [26]:
def truncate_by_turns(history, n_turns=5):
    """
    Keeps the last n turns of the conversation.
    Note: A 'turn' consists of one user message and one assistant message.
    So, n_turns=5 means keeping the last 10 messages (5 user, 5 assistant).
    """
    return history[-(n_turns * 2):]

def truncate_by_length(history, max_length=2000):
    """
    Keeps the most recent messages that fit within a character limit.
    """
    current_length = 0
    truncated_history = []
    # Iterate backwards through history to keep the most recent messages first
    for message in reversed(history):
        message_length = len(message['content'])
        if current_length + message_length <= max_length:
            truncated_history.insert(0, message)
            current_length += message_length
        else:
            break
    return truncated_history

def summarize_conversation(history):
    """
    Uses the Groq API to summarize the conversation history.
    """
    # Combine the conversation into a single string for the model
    conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])

    print("\n⏳ Summarizing conversation...")
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a summarization expert. Your task is to create a concise summary of the following conversation. The summary should capture the key topics, decisions, and outcomes. It will be used as a memory for the next conversation, so make it informative but brief."
                },
                {
                    "role": "user",
                    "content": conversation_text
                }
            ],
            model=CHAT_MODEL,
        )
        summary = chat_completion.choices[0].message.content
        print("✅ Summarization complete.")
        return summary
    except Exception as e:
        print(f"An error occurred during summarization: {e}")
        return "Summary could not be generated."

# 2. Demonstration

Now, let's simulate a conversation and see how summarization and truncation work. We will set the summarization to trigger after every 3rd run (i.e., after the 3rd user-assistant exchange).

In [27]:
# --- Configuration ---
PERIODIC_SUMMARIZATION_K = 3 # Summarize after every 3 runs

# --- Initialization ---
conversation_history = []
run_counter = 0

# --- Sample Conversation Data ---
sample_chats = [
    "Hi there! I'm planning a trip to Japan. Can you give me some ideas for a 7-day itinerary?",
    "That sounds great! For the first part of the trip, let's focus on Tokyo. What are the must-see spots there?",
    "I love anime, so Akihabara is a must. I also enjoy historical sites. Let's add the Senso-ji Temple. For the third spot, how about the Shibuya Crossing? Now, what about day trips from Tokyo?",
    "Hakone sounds perfect. I'd love to see Mt. Fuji. Let's add that to the plan. So, the first 4 days are set. What about the remaining 3 days? I was thinking of visiting Kyoto.",
    "Excellent. In Kyoto, I want to see the Fushimi Inari Shrine and Kinkaku-ji. What's a good third option?",
    "Arashiyama Bamboo Grove it is. This sounds like a fantastic plan. Can you please give me a final overview of the 7-day itinerary?"
]

# --- Main Conversation Loop ---
for user_input in sample_chats:
    run_counter += 1
    print(f"\n\n--- 🚀 Run #{run_counter} ---")

    # 1. Add user message to history
    conversation_history.append({"role": "user", "content": user_input})
    print(f"👤 User: {user_input}")

    # 2. Get assistant's response
    # In a real app, we would truncate the history before sending it to the model
    # to manage token limits. For this demo, we'll show the effects later.
    chat_completion = client.chat.completions.create(
        messages=conversation_history,
        model=CHAT_MODEL,
    )
    assistant_response = chat_completion.choices[0].message.content

    # 3. Add assistant message to history
    conversation_history.append({"role": "assistant", "content": assistant_response})
    print(f"🤖 Assistant: {assistant_response}")

    # 4. Perform periodic summarization
    if run_counter % PERIODIC_SUMMARIZATION_K == 0:
        print(f"\n🔥 Triggering periodic summarization after {run_counter} runs...")
        # Get the summary of the current history
        summary = summarize_conversation(conversation_history)

        # Create a new history: summary + the last conversation turn for context
        new_history = [
            {"role": "system", "content": f"This is a summary of the previous conversation: {summary}"},
            conversation_history[-2], # Last user message
            conversation_history[-1], # Last assistant message
        ]
        conversation_history = new_history
        print("\n✨ Conversation history has been replaced with its summary.")
        print(f"New History:\n{json.dumps(conversation_history, indent=2)}")


# --- Final History ---
print("\n\n--- ✅ Final Conversation History ---")
print(json.dumps(conversation_history, indent=2))


# --- Demonstrating Truncation Options on the Final History ---

print("\n\n--- ✂️ Demonstrating Truncation ---")

# 1. Truncate by number of turns (e.g., last 1 turn)
truncated_by_turns_history = truncate_by_turns(conversation_history, n_turns=1)
print(f"\n1. History Truncated to the Last 1 Turn:")
print(json.dumps(truncated_by_turns_history, indent=2))


# 2. Truncate by character length (e.g., last 1000 characters)
truncated_by_length_history = truncate_by_length(conversation_history, max_length=1000)
print(f"\n2. History Truncated to a Max Length of 1000 Characters:")
print(json.dumps(truncated_by_length_history, indent=2))



--- 🚀 Run #1 ---
👤 User: Hi there! I'm planning a trip to Japan. Can you give me some ideas for a 7-day itinerary?
🤖 Assistant: Japan is a wonderful destination with so much to see, experience, and explore. Here's a suggested 7-day itinerary for you:

**Day 1: Arrival and Exploration of Tokyo**

* Arrive at Narita or Haneda airport
* Take a train or bus to your hotel in Tokyo
* Visit the famous Shibuya Crossing and take a walk around the Shibuya area
* Explore the trendy Harajuku district and see the fashionable youth culture
* Try some delicious food, such as Tonkatsu or Ramen, at a local restaurant

**Day 2: Tokyo Landmarks**

* Visit the Tokyo Skytree for panoramic views of the city
* Explore the Asakusa district, including Senso-ji Temple, one of the oldest temples in Japan
* Visit the Meiji Shrine, a beautiful Shinto shrine dedicated to the deified spirits of Emperor Meiji and his wife, Empress Shoken
* Take a stroll through the beautiful Imperial Palace East Garden

**Day 3: Tr

# **Task 2: JSON Schema Classification & Information Extraction**

This task demonstrates how to use Groq's function calling capabilities (compatible with the OpenAI SDK) to extract structured data from a user's message based on a predefined JSON schema.

# 1. JSON Schema Definition
We will define a schema for a function called extract_user_info. The model will be instructed to "call" this function and provide arguments that match the user's input.

In [28]:
# Define the JSON schema for extracting user details.
# This structure tells the model what information to find and in what format.
user_details_schema = {
    "type": "function",
    "function": {
        "name": "extract_user_info",
        "description": "Extracts key details about a user from their message and stores them.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The user's full name, e.g., 'John Doe'."
                },
                "email": {
                    "type": "string",
                    "description": "The user's email address, e.g., 'user@example.com'."
                },
                "phone": {
                    "type": "string",
                    "description": "The user's phone number, including country code if provided."
                },
                "location": {
                    "type": "string",
                    "description": "The user's city and country, e.g., 'San Francisco, USA'."
                },
                "age": {
                    "type": "integer",
                    "description": "The user's age in years."
                },
            },
            # We can specify which fields are mandatory for the function call.
            "required": ["name", "email"]
        },
    }
}

# 2. Demonstration

Let's test the schema with a few sample chats. We will create a helper function to process the chat and validate the output.

In [29]:
def extract_details_from_chat(user_prompt):
    """
    Uses Groq's function calling to extract structured data from a prompt.
    """
    print(f"\n--- 📝 Parsing Chat ---")
    print(f"Input: \"{user_prompt}\"")

    try:
        chat_completion = client.chat.completions.create(
            model=EXTRACTION_MODEL,
            messages=[{"role": "user", "content": user_prompt}],
            tools=[user_details_schema],
            # We force the model to use our function
            tool_choice={"type": "function", "function": {"name": "extract_user_info"}},
        )

        # --- Extract and Validate the Output ---
        tool_call = chat_completion.choices[0].message.tool_calls[0]
        if tool_call.function.name == "extract_user_info":
            arguments_str = tool_call.function.arguments
            extracted_data = json.loads(arguments_str)

            print("\n✅ Successfully Extracted JSON:")
            print(json.dumps(extracted_data, indent=2))

            # Basic Validation
            print("\n🔍 Validation Results:")
            schema_props = user_details_schema['function']['parameters']['properties']
            for key, value in extracted_data.items():
                expected_type = schema_props.get(key, {}).get('type')
                if expected_type:
                    # Simple type checking
                    is_valid = False
                    if expected_type == 'string' and isinstance(value, str):
                        is_valid = True
                    elif expected_type == 'integer' and isinstance(value, int):
                        is_valid = True

                    if is_valid:
                        print(f"  - '{key}': Valid (Type: {expected_type})")
                    else:
                        print(f"  - '{key}': ⚠️ Invalid! Expected {expected_type}, got {type(value).__name__}.")
            return extracted_data
        else:
            print("❌ Error: The model did not call the expected function.")
            return None

    except Exception as e:
        print(f"An error occurred during extraction: {e}")
        return None


# --- Sample Chats for Extraction ---
sample_extraction_chats = [
    "Hi, my name is Jane Doe and I'm 28. My email is jane.d@email.com and you can reach me at 123-456-7890. I live in New York.",
    "Please sign me up. I'm Michael Smith, my contact email is m.smith@workplace.net. I'm based in London, UK. My phone number is +44 20 7946 0958.",
    "Okay, I'm ready. Use this email: contact@domain.org. The name is Alex Ray. Age: 35. Phone: 9876543210. Location: Sydney.",
]

# --- Process each sample chat ---
for chat in sample_extraction_chats:
    extract_details_from_chat(chat)


--- 📝 Parsing Chat ---
Input: "Hi, my name is Jane Doe and I'm 28. My email is jane.d@email.com and you can reach me at 123-456-7890. I live in New York."

✅ Successfully Extracted JSON:
{
  "age": 28,
  "email": "jane.d@email.com",
  "location": "New York",
  "name": "Jane Doe",
  "phone": "123-456-7890"
}

🔍 Validation Results:
  - 'age': Valid (Type: integer)
  - 'email': Valid (Type: string)
  - 'location': Valid (Type: string)
  - 'name': Valid (Type: string)
  - 'phone': Valid (Type: string)

--- 📝 Parsing Chat ---
Input: "Please sign me up. I'm Michael Smith, my contact email is m.smith@workplace.net. I'm based in London, UK. My phone number is +44 20 7946 0958."

✅ Successfully Extracted JSON:
{
  "email": "m.smith@workplace.net",
  "location": "London, UK",
  "name": "Michael Smith",
  "phone": "+44 20 7946 0958"
}

🔍 Validation Results:
  - 'email': Valid (Type: string)
  - 'location': Valid (Type: string)
  - 'name': Valid (Type: string)
  - 'phone': Valid (Type: string)

-